# SQL เชิงลึกสำหรับการวิเคราะห์ข้อมูล

## วัตถุประสงค์การเรียนรู้
- เข้าใจการใช้ JOIN เพื่อเชื่อมตาราง
- สามารถใช้ฟังก์ชัน Aggregation (SUM, AVG, COUNT)
- เรียนรู้การสร้าง Subqueries
- ประยุกต์ใช้ SQL ในการวิเคราะห์ข้อมูลจริง

## 1. แนะนำ SQL สำหรับการวิเคราะห์ข้อมูล

SQL (Structured Query Language) เป็นภาษามาตรฐานสำหรับการจัดการและดึงข้อมูลจากฐานข้อมูลเชิงสัมพันธ์ (Relational Database) ในบทนี้เราจะเรียนรู้การใช้ SQL ในการวิเคราะห์ข้อมูลเชิงลึก

เราจะใช้ `sqlite3` ซึ่งเป็นไลบรารีที่มาพร้อมกับ Python เพื่อทำงานกับฐานข้อมูล SQLite

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# สร้างฐานข้อมูล in-memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("เชื่อมต่อกับฐานข้อมูล SQLite เรียบร้อยแล้ว")

## 2. การสร้างและนำเข้าข้อมูลตัวอย่าง

เราจะสร้างตารางข้อมูลจำลองเพื่อใช้ในการเรียนรู้ SQL เชิงลึก รวมทั้งสร้างความสัมพันธ์ระหว่างตาราง

In [ ]:
# สร้างตาราง customers (ลูกค้า)
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    city TEXT,
    registration_date DATE
);
''')

# สร้างตาราง products (สินค้า)
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL,
    stock_quantity INTEGER
);
''')

# สร้างตาราง orders (คำสั่งซื้อ)
cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount REAL,
    status TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
''')

# สร้างตาราง order_items (รายการสินค้าในคำสั่งซื้อ)
cursor.execute('''
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price REAL,
    FOREIGN KEY (order_id) REFERENCES orders (order_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
);
''')

print("สร้างตารางเรียบร้อยแล้ว")

### 2.1 เพิ่มข้อมูลตัวอย่าง

In [ ]:
# เพิ่มข้อมูลลูกค้า
customers_data = [
    (1, 'สมชาย', 'รักเรียน', 'somchai@email.com', '081-111-1111', 'กรุงเทพฯ', '2023-01-15'),
    (2, 'วิชัย', 'ใจดี', 'wichai@email.com', '082-222-2222', 'เชียงใหม่', '2023-02-20'),
    (3, 'มานี', 'มีทรัพย์', 'manee@email.com', '083-333-3333', 'ขอนแก่น', '2023-03-10'),
    (4, 'ประหยัด', 'จัดเต็ม', 'prayad@email.com', '084-444-4444', 'กรุงเทพฯ', '2023-04-05'),
    (5, 'สุชาติ', 'ชาตินิยม', 'suchart@email.com', '085-555-5555', 'ภูเก็ต', '2023-05-12'),
    (6, 'พิมพา', 'น่ารัก', 'pimpa@email.com', '086-666-6666', 'เชียงใหม่', '2023-06-18'),
    (7, 'กัญญา', 'รักสวย', 'kanya@email.com', '087-777-7777', 'กรุงเทพฯ', '2023-07-22'),
    (8, 'อนุชา', 'ชาญฉลาด', 'anucha@email.com', '088-888-8888', 'อยุธยา', '2023-08-30'),
    (9, 'อภิชาติ', 'ชาติชาย', 'apichart@email.com', '089-999-9999', 'กรุงเทพฯ', '2023-09-14'),
    (10, 'รัตนา', 'น่าชื่นชม', 'rattana@email.com', '090-000-0000', 'เชียงใหม่', '2023-10-25')
]

cursor.executemany('''
INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?, ?)
''', customers_data)

# เพิ่มข้อมูลสินค้า
products_data = [
    (1, 'สมาร์ทโฟน รุ่น X', 'อิเล็กทรอนิกส์', 25000, 50),
    (2, 'โน้ตบุ๊ก รุ่น Y', 'อิเล็กทรอนิกส์', 35000, 30),
    (3, 'แท็บเล็ต รุ่น Z', 'อิเล็กทรอนิกส์', 15000, 40),
    (4, 'เสื้อยืด ลาย A', 'เสื้อผ้า', 350, 200),
    (5, 'กางเกงยีนส์ ทรง B', 'เสื้อผ้า', 990, 150),
    (6, 'รองเท้าวิ่ง รุ่น C', 'รองเท้า', 2500, 100),
    (7, 'กระเป๋าเป้ รุ่น D', 'กระเป๋า', 1200, 80),
    (8, 'หูฟังไร้สาย รุ่น E', 'อิเล็กทรอนิกส์', 3500, 60),
    (9, 'สายชาร์จ รุ่น F', 'อิเล็กทรอนิกส์', 250, 300),
    (10, 'เครื่องฟอกอากาศ รุ่น G', 'เครื่องใช้ไฟฟ้า', 8000, 20)
]

cursor.executemany('''
INSERT INTO products VALUES (?, ?, ?, ?, ?)
''', products_data)

# เพิ่มข้อมูลคำสั่งซื้อ
orders_data = [
    (1, 1, '2024-01-10', 25000, 'ส่งแล้ว'),
    (2, 2, '2024-01-15', 35000, 'ส่งแล้ว'),
    (3, 3, '2024-01-20', 15350, 'ส่งแล้ว'),
    (4, 1, '2024-02-05', 3490, 'ส่งแล้ว'),
    (5, 5, '2024-02-12', 8000, 'ส่งแล้ว'),
    (6, 6, '2024-02-18', 3700, 'กำลังเตรียมการ'),
    (7, 7, '2024-02-25', 25250, 'ส่งแล้ว'),
    (8, 8, '2024-03-02', 15000, 'กำลังเตรียมการ'),
    (9, 9, '2024-03-08', 35000, 'รอชำระเงิน'),
    (10, 10, '2024-03-15', 8000, 'รอชำระเงิน'),
    (11, 1, '2024-03-20', 3750, 'กำลังเตรียมการ'),
    (12, 2, '2024-03-25', 15990, 'กำลังเตรียมการ'),
    (13, 3, '2024-04-01', 25000, 'รอชำระเงิน'),
    (14, 5, '2024-04-05', 3500, 'รอชำระเงิน'),
    (15, 7, '2024-04-10', 35000, 'รอชำระเงิน')
]

cursor.executemany('''
INSERT INTO orders VALUES (?, ?, ?, ?, ?)
''', orders_data)

# เพิ่มข้อมูลรายการสินค้าในคำสั่งซื้อ
order_items_data = [
    (1, 1, 1, 1, 25000),
    (2, 2, 2, 1, 35000),
    (3, 3, 3, 1, 15000),
    (4, 3, 4, 1, 350),
    (5, 4, 4, 5, 350),
    (6, 4, 5, 2, 990),
    (7, 5, 10, 1, 8000),
    (8, 6, 8, 1, 3500),
    (9, 6, 9, 1, 200),
    (10, 7, 1, 1, 25000),
    (11, 7, 9, 1, 250),
    (12, 8, 3, 1, 15000),
    (13, 9, 2, 1, 35000),
    (14, 10, 10, 1, 8000),
    (15, 11, 6, 1, 2500),
    (16, 11, 8, 1, 1250),
    (17, 12, 3, 1, 15000),
    (18, 12, 5, 1, 990),
    (19, 13, 1, 1, 25000),
    (20, 14, 8, 1, 3500),
    (21, 15, 2, 1, 35000)
]

cursor.executemany('''
INSERT INTO order_items VALUES (?, ?, ?, ?, ?)
''', order_items_data)

conn.commit()
print("เพิ่มข้อมูลเรียบร้อยแล้ว")

### 2.2 ทดสอบดูข้อมูลในตาราง

In [ ]:
# ฟังก์ชันสำหรับดึงข้อมูลจาก SQL และแสดงผลเป็น DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

# ดูข้อมูลในตาราง customers
print("ข้อมูลในตาราง customers:")
display(run_query("SELECT * FROM customers LIMIT 5"))

# ดูข้อมูลในตาราง products
print("\nข้อมูลในตาราง products:")
display(run_query("SELECT * FROM products LIMIT 5"))

# ดูข้อมูลในตาราง orders
print("\nข้อมูลในตาราง orders:")
display(run_query("SELECT * FROM orders LIMIT 5"))

# ดูข้อมูลในตาราง order_items
print("\nข้อมูลในตาราง order_items:")
display(run_query("SELECT * FROM order_items LIMIT 5"))

## 3. การใช้ JOIN เพื่อเชื่อมตาราง

การใช้ JOIN เป็นเทคนิคสำคัญในการวิเคราะห์ข้อมูลจากหลายตาราง มีหลายประเภทของ JOIN ที่ใช้ในการเชื่อมต่อข้อมูล:

1. **INNER JOIN**: รวมข้อมูลที่มีค่าตรงกันในทั้งสองตาราง
2. **LEFT JOIN**: รวมทุกข้อมูลจากตารางทางซ้าย และข้อมูลที่ตรงกันจากตารางทางขวา
3. **RIGHT JOIN**: รวมทุกข้อมูลจากตารางทางขวา และข้อมูลที่ตรงกันจากตารางทางซ้าย
4. **FULL OUTER JOIN**: รวมทุกข้อมูลจากทั้งสองตาราง

### 3.1 INNER JOIN

INNER JOIN ใช้เพื่อรวมข้อมูลจากสองตารางที่มีค่าตรงกันตามเงื่อนไข

In [ ]:
# ค้นหาข้อมูลคำสั่งซื้อพร้อมข้อมูลลูกค้า
query = '''
SELECT 
    o.order_id, 
    o.order_date, 
    o.total_amount, 
    o.status, 
    c.first_name, 
    c.last_name, 
    c.city
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
ORDER BY o.order_date DESC
LIMIT 10
'''

result = run_query(query)
print("คำสั่งซื้อพร้อมข้อมูลลูกค้า (INNER JOIN):")
display(result)

### 3.2 LEFT JOIN

LEFT JOIN ใช้เพื่อรวมทุกข้อมูลจากตารางทางซ้าย และข้อมูลที่ตรงกันจากตารางทางขวา

In [ ]:
# ค้นหาลูกค้าทั้งหมดพร้อมคำสั่งซื้อถ้ามี (แสดงลูกค้าทุกคนแม้ไม่มีคำสั่งซื้อ)
query = '''
SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    c.city, 
    o.order_id, 
    o.order_date, 
    o.total_amount
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
ORDER BY c.customer_id
'''

result = run_query(query)
print("ลูกค้าทั้งหมดพร้อมคำสั่งซื้อ (LEFT JOIN):")
display(result)

### 3.3 การเชื่อมตารางมากกว่า 2 ตาราง

เราสามารถเชื่อมต่อข้อมูลจากหลายตารางได้ในคำสั่ง SQL เดียว

In [ ]:
# ค้นหาข้อมูลรายละเอียดคำสั่งซื้อพร้อมข้อมูลลูกค้าและสินค้า
query = '''
SELECT 
    o.order_id, 
    o.order_date, 
    c.first_name, 
    c.last_name, 
    p.product_name, 
    oi.quantity, 
    oi.unit_price, 
    (oi.quantity * oi.unit_price) AS subtotal
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_date DESC, o.order_id
LIMIT 15
'''

result = run_query(query)
print("รายละเอียดคำสั่งซื้อพร้อมข้อมูลลูกค้าและสินค้า:")
display(result)

## 4. ฟังก์ชัน Aggregation (SUM, AVG, COUNT, MIN, MAX)

ฟังก์ชัน Aggregation ใช้ในการคำนวณค่าสรุปของข้อมูล เช่น ผลรวม, ค่าเฉลี่ย, จำนวน, ค่าต่ำสุด, ค่าสูงสุด เป็นต้น

In [ ]:
# ค้นหาจำนวนคำสั่งซื้อ, ยอดขายรวม, ยอดขายเฉลี่ย, ยอดขายต่ำสุด, และยอดขายสูงสุด
query = '''
SELECT 
    COUNT(order_id) AS total_orders,
    SUM(total_amount) AS total_sales,
    AVG(total_amount) AS average_order_value,
    MIN(total_amount) AS min_order_value,
    MAX(total_amount) AS max_order_value
FROM orders
'''

result = run_query(query)
print("สรุปข้อมูลคำสั่งซื้อ:")
display(result)

### 4.1 การใช้ GROUP BY

คำสั่ง GROUP BY ใช้ในการจัดกลุ่มข้อมูลตามค่าในคอลัมน์หนึ่งหรือหลายคอลัมน์ และคำนวณค่าสรุปสำหรับแต่ละกลุ่ม

In [ ]:
# ค้นหายอดขายรวมตามเมือง
query = '''
SELECT 
    c.city, 
    COUNT(o.order_id) AS order_count,
    SUM(o.total_amount) AS total_sales
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.city
ORDER BY total_sales DESC
'''

result = run_query(query)
print("ยอดขายรวมตามเมือง:")
display(result)

# สร้างกราฟแสดงยอดขายตามเมือง
plt.figure(figsize=(10, 6))
plt.bar(result['city'], result['total_sales'])
plt.title('ยอดขายรวมตามเมือง')
plt.xlabel('เมือง')
plt.ylabel('ยอดขายรวม (บาท)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# ค้นหายอดขายตามหมวดหมู่สินค้า
query = '''
SELECT 
    p.category, 
    COUNT(oi.order_item_id) AS item_count,
    SUM(oi.quantity * oi.unit_price) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.category
ORDER BY total_sales DESC
'''

result = run_query(query)
print("ยอดขายตามหมวดหมู่สินค้า:")
display(result)

# สร้างกราฟวงกลมแสดงสัดส่วนยอดขายตามหมวดหมู่สินค้า
plt.figure(figsize=(8, 8))
plt.pie(result['total_sales'], labels=result['category'], autopct='%1.1f%%', startangle=90, shadow=True)
plt.title('สัดส่วนยอดขายตามหมวดหมู่สินค้า')
plt.axis('equal')  # เพื่อให้วงกลมเป็นวงกลมจริงๆ
plt.tight_layout()
plt.show()

### 4.2 การใช้ HAVING

คำสั่ง HAVING ใช้ในการกรองผลลัพธ์หลังจากที่ GROUP BY ได้จัดกลุ่มข้อมูลแล้ว (คล้ายกับ WHERE แต่ใช้กับผลลัพธ์จาก GROUP BY)

In [ ]:
# ค้นหาลูกค้าที่มียอดสั่งซื้อรวมมากกว่า 50,000 บาท
query = '''
SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    COUNT(o.order_id) AS order_count,
    SUM(o.total_amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING SUM(o.total_amount) > 50000
ORDER BY total_spent DESC
'''

result = run_query(query)
print("ลูกค้าที่มียอดสั่งซื้อรวมมากกว่า 50,000 บาท:")
display(result)

## 5. การใช้ Subqueries (การใช้คำสั่ง SQL ซ้อนกัน)

Subqueries คือการใช้คำสั่ง SQL ซ้อนกันภายในคำสั่ง SQL อื่น ซึ่งทำให้เราสามารถทำการวิเคราะห์ข้อมูลที่ซับซ้อนมากขึ้น

In [ ]:
# ค้นหาสินค้าที่มีราคาสูงกว่าราคาเฉลี่ยของสินค้าทั้งหมด
query = '''
SELECT 
    product_id, 
    product_name, 
    category, 
    price
FROM products
WHERE price > (SELECT AVG(price) FROM products)
ORDER BY price DESC
'''

result = run_query(query)
print("สินค้าที่มีราคาสูงกว่าราคาเฉลี่ย:")
display(result)

In [ ]:
# ค้นหาลูกค้าที่มีคำสั่งซื้อในเดือนล่าสุด
query = '''
SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    c.email, 
    c.city
FROM customers c
WHERE c.customer_id IN (
    SELECT DISTINCT customer_id
    FROM orders
    WHERE order_date >= (SELECT MAX(order_date) FROM orders) - 30
)
ORDER BY c.customer_id
'''

result = run_query(query)
print("ลูกค้าที่มีคำสั่งซื้อในเดือนล่าสุด:")
display(result)

### 5.1 Correlated Subqueries

Correlated Subqueries คือ subqueries ที่อ้างอิงไปยังตารางในคำสั่ง query ภายนอก

In [ ]:
# ค้นหาคำสั่งซื้อที่มีมูลค่าสูงกว่าค่าเฉลี่ยของลูกค้ารายนั้น
query = '''
SELECT 
    o.order_id, 
    o.customer_id, 
    c.first_name, 
    c.last_name, 
    o.order_date, 
    o.total_amount
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.total_amount > (
    SELECT AVG(total_amount)
    FROM orders
    WHERE customer_id = o.customer_id
)
ORDER BY o.customer_id, o.order_date
'''

result = run_query(query)
print("คำสั่งซื้อที่มีมูลค่าสูงกว่าค่าเฉลี่ยของลูกค้ารายนั้น:")
display(result)

## 6. Common Table Expressions (CTEs)

CTEs ช่วยให้เราสามารถสร้าง "ตารางชั่วคราว" เพื่อใช้ในการ query ที่ซับซ้อน ทำให้โค้ดอ่านง่ายขึ้นและลดความซับซ้อน

In [ ]:
# ใช้ CTE เพื่อหาลูกค้าที่มียอดซื้อสูงที่สุด 3 อันดับแรก
query = '''
WITH CustomerSales AS (
    SELECT 
        c.customer_id, 
        c.first_name, 
        c.last_name, 
        c.city, 
        SUM(o.total_amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.city
)
SELECT *
FROM CustomerSales
ORDER BY total_spent DESC
LIMIT 3
'''

result = run_query(query)
print("ลูกค้าที่มียอดซื้อสูงที่สุด 3 อันดับแรก:")
display(result)

In [ ]:
# ใช้หลาย CTEs เพื่อวิเคราะห์ข้อมูลที่ซับซ้อน
query = '''
WITH MonthlySales AS (
    SELECT 
        strftime('%Y-%m', order_date) AS month,
        SUM(total_amount) AS monthly_total
    FROM orders
    GROUP BY month
),
CategorySales AS (
    SELECT 
        strftime('%Y-%m', o.order_date) AS month,
        p.category,
        SUM(oi.quantity * oi.unit_price) AS category_total
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    GROUP BY month, p.category
)
SELECT 
    cs.month, 
    cs.category, 
    cs.category_total,
    ms.monthly_total,
    ROUND(cs.category_total * 100.0 / ms.monthly_total, 2) AS percentage
FROM CategorySales cs
JOIN MonthlySales ms ON cs.month = ms.month
ORDER BY cs.month, percentage DESC
'''

result = run_query(query)
print("สัดส่วนยอดขายตามหมวดหมู่สินค้าในแต่ละเดือน:")
display(result)

## 7. การสร้างวิว (Views)

วิว (View) คือคำสั่ง SELECT ที่ถูกเก็บไว้ในฐานข้อมูลเพื่อใช้งานซ้ำ ช่วยให้เราสามารถเข้าถึงข้อมูลที่ซับซ้อนได้อย่างสะดวก

In [ ]:
# สร้างวิวสำหรับข้อมูลลูกค้าและยอดซื้อ
cursor.execute('''
CREATE VIEW customer_sales_summary AS
SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    c.city, 
    COUNT(o.order_id) AS order_count,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.city
''')

print("สร้างวิว customer_sales_summary เรียบร้อยแล้ว")

# ใช้วิวที่สร้างขึ้น
result = run_query("SELECT * FROM customer_sales_summary ORDER BY total_spent DESC")
print("\nข้อมูลจากวิว customer_sales_summary:")
display(result)

## 8. การใช้ Window Functions

Window Functions เป็นฟังก์ชันที่ทำงานบน "หน้าต่าง" (window) ของแถวที่เกี่ยวข้องกับแถวปัจจุบัน ใช้สำหรับการวิเคราะห์ข้อมูลขั้นสูง

In [ ]:
# ใช้ Window Function เพื่อคำนวณอันดับของลูกค้าตามยอดซื้อ
query = '''
SELECT 
    customer_id, 
    first_name, 
    last_name, 
    city, 
    total_spent,
    RANK() OVER (ORDER BY total_spent DESC) AS sales_rank
FROM customer_sales_summary
WHERE total_spent > 0
'''

result = run_query(query)
print("อันดับของลูกค้าตามยอดซื้อ:")
display(result)

In [ ]:
# ใช้ Window Function เพื่อหายอดขายสะสมตามเวลา
query = '''
SELECT 
    order_date, 
    total_amount,
    SUM(total_amount) OVER (ORDER BY order_date) AS running_total
FROM orders
ORDER BY order_date
'''

result = run_query(query)
print("ยอดขายสะสมตามเวลา:")
display(result)

# สร้างกราฟแสดงยอดขายสะสมตามเวลา
plt.figure(figsize=(12, 6))
plt.plot(result['order_date'], result['running_total'], marker='o', linestyle='-', color='blue')
plt.title('ยอดขายสะสมตามเวลา')
plt.xlabel('วันที่')
plt.ylabel('ยอดขายสะสม (บาท)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. แบบฝึกหัด: การวิเคราะห์ข้อมูลด้วย SQL

ให้ใช้ความรู้เกี่ยวกับ SQL ที่ได้เรียนมาเพื่อทำการวิเคราะห์ข้อมูลต่อไปนี้:

### แบบฝึกหัดที่ 1: รายงานยอดขายรายเดือน

สร้างรายงานแสดงยอดขายรายเดือน จำนวนคำสั่งซื้อ และมูลค่าเฉลี่ยต่อคำสั่งซื้อ เรียงตามเดือน

In [ ]:
# เพิ่มโค้ดของคุณที่นี่
query = '''
-- เขียนคำสั่ง SQL ของคุณที่นี่
'''

result = run_query(query)
print("รายงานยอดขายรายเดือน:")
display(result)

# สร้างกราฟแสดงผล (ถ้าต้องการ)

### แบบฝึกหัดที่ 2: ค้นหาสินค้าขายดี

ค้นหาสินค้า 5 อันดับแรกที่มียอดขายสูงสุด (จำนวนชิ้น × ราคาต่อชิ้น) พร้อมแสดงหมวดหมู่และจำนวนที่ขายได้

In [ ]:
# เพิ่มโค้ดของคุณที่นี่
query = '''
-- เขียนคำสั่ง SQL ของคุณที่นี่
'''

result = run_query(query)
print("สินค้า 5 อันดับแรกที่มียอดขายสูงสุด:")
display(result)

# สร้างกราฟแสดงผล (ถ้าต้องการ)

### แบบฝึกหัดที่ 3: ค้นหาลูกค้าที่ไม่มีคำสั่งซื้อในช่วง 30 วันล่าสุด

ค้นหารายชื่อลูกค้าที่ไม่มีคำสั่งซื้อในช่วง 30 วันล่าสุด เรียงตามวันที่สั่งซื้อล่าสุด (จากเก่าสุดไปใหม่สุด)

In [ ]:
# เพิ่มโค้ดของคุณที่นี่
query = '''
-- เขียนคำสั่ง SQL ของคุณที่นี่
'''

result = run_query(query)
print("ลูกค้าที่ไม่มีคำสั่งซื้อในช่วง 30 วันล่าสุด:")
display(result)

### แบบฝึกหัดที่ 4: วิเคราะห์ยอดขายตามเมืองและหมวดหมู่สินค้า

สร้างรายงานแสดงยอดขายตามเมืองและหมวดหมู่สินค้า เพื่อวิเคราะห์ว่าลูกค้าในแต่ละเมืองมีความสนใจในสินค้าหมวดหมู่ใดบ้าง

In [ ]:
# เพิ่มโค้ดของคุณที่นี่
query = '''
-- เขียนคำสั่ง SQL ของคุณที่นี่
'''

result = run_query(query)
print("ยอดขายตามเมืองและหมวดหมู่สินค้า:")
display(result)

# สร้างกราฟแสดงผล (ถ้าต้องการ)

## 10. สรุป

ในบทนี้ เราได้เรียนรู้เกี่ยวกับ SQL เชิงลึกสำหรับการวิเคราะห์ข้อมูล ซึ่งรวมถึง:

1. **การใช้ JOIN เพื่อเชื่อมตาราง** - เรียนรู้วิธีการรวมข้อมูลจากหลายตารางด้วย INNER JOIN, LEFT JOIN และการเชื่อมตารางมากกว่า 2 ตาราง

2. **ฟังก์ชัน Aggregation** - เรียนรู้การใช้ฟังก์ชัน SUM, AVG, COUNT, MIN, MAX และการใช้ GROUP BY และ HAVING

3. **Subqueries** - เรียนรู้การใช้คำสั่ง SQL ซ้อนกันเพื่อวิเคราะห์ข้อมูลที่ซับซ้อน

4. **Common Table Expressions (CTEs)** - เรียนรู้การสร้าง "ตารางชั่วคราว" เพื่อใช้ในการ query ที่ซับซ้อน

5. **การสร้างวิว (Views)** - เรียนรู้การสร้างวิวเพื่อใช้งานคำสั่ง SQL ซ้ำ

6. **Window Functions** - เรียนรู้การใช้ฟังก์ชันที่ทำงานบน "หน้าต่าง" ของแถวที่เกี่ยวข้องกับแถวปัจจุบัน

ทักษะเหล่านี้เป็นพื้นฐานสำคัญสำหรับการวิเคราะห์ข้อมูลในโปรเจ็กต์ Data Science และ Data Analysis และช่วยให้สามารถดึงข้อมูลที่ต้องการจากฐานข้อมูลได้อย่างมีประสิทธิภาพ